In [ ]:
# Lazify Samples

In [3]:
import json, tqdm, random, concurrent
from datasets import load_dataset
from llms import generate_json
from functools import partial

with open("prompts/gsm8k_sharding_segmentation.txt", "r") as f:
    prompt_segment = f.read()

with open("prompts/gsm8k_sharding_conversational.txt", "r") as f:
    prompt_conversational = f.read()

with open("prompts/sharding_verification.txt", "r") as f:
    prompt_verification = f.read()

MODEL = "t-o3"

def shard_gsm8k(seed):

    prompt_extractive_populated = prompt_segment.replace("[[QUESTION]]", seed["question"])

    verification_status = None
    attempt = 0
    while verification_status != "complete":
        attempt += 1
        if attempt > 5:
            print(f"Failed to get valid segments after 5 attempts. Skipping {seed['id']}")
            break
        try:
            segments_res = generate_json([{"role": "user", "content": prompt_extractive_populated}], model=MODEL, step="sharding-gsm8k-extractive")

            # Now do the conversational version
            prompt_conversational_populated = prompt_conversational.replace("[[QUESTION]]", seed["question"]).replace("[[SEGMENTS]]", json.dumps(segments_res["segments"]))
            conversational_res = generate_json([{"role": "user", "content": prompt_conversational_populated}], model=MODEL, step="sharding-gsm8k-conversational")


            # Assign hint_id per item in hints, starting from 1
            shard_attempt = [{"shard_id": 1, "shard": conversational_res["initial_query"]}]
            for i, h in enumerate(conversational_res["hints"]):
                shard_attempt.append({"shard_id": i + 2, "shard": h["hint"]})

            # Now do the verification
            prompt_verification_populated = prompt_verification.replace("[[QUERY]]", seed["question"]).replace("[[SHARDS]]", json.dumps(shard_attempt))
            verification_res = generate_json([{"role": "user", "content": prompt_verification_populated}], model=MODEL, step="sharding-gsm8k-verification")

            verification_status = verification_res["coverage"]
        except Exception as e:
            print("Fatal error when generating segments: ", e)
            continue

    shards = [{"shard_id": 1, "shard": conversational_res["initial_query"]}]
    for i, h in enumerate(conversational_res["hints"]):
        shards.append({"shard_id": i + 2, "shard": h["hint"]})

    seed["task_id"] = f"sharded-gsm8k-train-{seed['id']}"
    seed["fully_specified_question"] = seed["question"]
    seed["shards"] = shards
    seed["model_name"] = MODEL

    return seed


def shard_data(output_path, shuffle=True, max_samples=500, num_workers=10):
    dataset = load_dataset("gsm8k", "main")
    seeds = list(dataset['train'])
    # add "id" to each sample
    for i, seed in enumerate(seeds):
        seed["id"] = i
        seed["task"] = "math"

    if shuffle:
        random.shuffle(seeds)
    
    seed_samples = seeds[:max_samples] if max_samples else seeds

    print("Total to convert: ", len(seed_samples))

    process_sample = partial(shard_gsm8k)

    # Process samples in parallel using ThreadPoolExecutor instead of ProcessPoolExecutor
    # This works better in Jupyter notebooks
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Use tqdm to show progress
        results = list(tqdm.tqdm(
            executor.map(process_sample, seed_samples),
            total=len(seed_samples),
            desc=f"Processing with {num_workers} threads"
        ))
    
    # Filter out None results and add to output data
    output_data = [result for result in results if result is not None and "shards" in result]
    print(f"Generated {len(output_data)} sharded instructions")

    with open(output_path, "w") as f:
        json.dump(output_data, f, indent=2)

    print(f"Processed {len(output_data)} samples successfully out of {len(seed_samples)} total samples")
    return output_data


In [5]:
converted = shard_data("data/sharded_gsm8k_train_400.json", shuffle=True, max_samples=400, num_workers=5)

Total to convert:  400


Processing with 5 threads:   0%|          | 2/400 [01:56<5:55:01, 53.52s/it]

Fatal error when generating segments:  list indices must be integers or slices, not str
Fatal error when generating segments:  list indices must be integers or slices, not str
Failed to get valid segments after 5 attempts. Skipping 404
Failed to get valid segments after 5 attempts. Skipping 84


Processing with 5 threads:   1%|          | 4/400 [04:58<8:27:32, 76.90s/it]

Failed to get valid segments after 5 attempts. Skipping 211


...

Failed to get valid segments after 5 attempts. Skipping 123


Processing with 5 threads:  97%|█████████▋| 387/400 [1:18:37<02:48, 12.99s/it]

Failed to get valid segments after 5 attempts. Skipping 1304


Processing with 5 threads: 100%|██████████| 400/400 [1:20:16<00:00, 12.04s/it]

Failed to get valid segments after 5 attempts. Skipping 1149
Generated 400 sharded instructions
Processed 400 samples successfully out of 400 total samples


In [4]:
import json
with open('data/sharded_gsm8k_train_400.json', 'r') as f:
    data = json.load(f)

# {'question': 'Saturday at the ice cream shop, there were twice as many people who ordered vanilla ice cream as ordered chocolate ice cream.  If 220 people ordered ice cream on Saturday, and 20% of those ordered vanilla ice cream, how many people ordered chocolate ice cream?', 'answer': 'If 20% of 220 customers ordered vanilla, then there were 0.2*220=<<0.2*220=44>>44 customers who ordered vanilla.\nIf there were twice as many people who ordered vanilla as chocolate, then 44/2=<<44/2=22>>22 customers ordered chocolate.\n#### 22', 'id': 2765, 'task': 'math', 'task_id': 'sharded-gsm8k-train-2765', 'fully_specified_question': 'Saturday at the ice cream shop, there were twice as many people who ordered vanilla ice cream as ordered chocolate ice cream.  If 220 people ordered ice cream on Saturday, and 20% of those ordered vanilla ice cream, how many people ordered chocolate ice cream?', 'shards': [{'shard_id': 2, 'shard': 'I see that twenty percent of the customers chose vanilla ice cream'}, {'shard_id': 3, 'shard': 'The vanilla orders are double the chocolate orders'}, {'shard_id': 1, 'shard': 'number of chocolate ice cream orders'}, {'shard_id': 4, 'shard': 'I know 220 people ordered ice cream on Saturday'}], 'model_name': 't-o3', 'verifications': {'full-avg': 1.0, 'full-all': {'t-gpt-4.1': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'concat-avg': 1.0, 'concat-all': {'t-gpt-4.1': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'shuffle-concat-avg': 1.0, 'shuffle-concat-all': {'t-gpt-4.1': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'sharded-avg': 0.875, 'sharded-all': {'t-gpt-4.1': [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]}}}
# select samples with above 0.7 on concat, full and shuffle-concat, and then any number between 0.0 < sharded < 1.0

for sample in data:
    validation_keys = ['full-avg', 'concat-avg', 'shuffle-concat-avg']
    is_valid = all(sample['verifications'][key] >= 0.7 for key in validation_keys)
    shard_ok = 0.0 < sample['verifications']['sharded-avg'] < 0.8
    sample["selected"] = is_valid and shard_ok

selected_samples = [sample for sample in data if sample["selected"]]

print(len(selected_samples))

# save the valid samples to a new file
with open('data/sharded_gsm8k_train_400_selected.json', 'w') as f:
    json.dump(selected_samples, f, indent=4)

67


In [2]:
# merge with code
from collections import Counter
import json

output_fn = "data/sharded_train_synthetic_0.1.json"
synthetic_fns = ["data/code_synthetic_problems_0.1_final_transformed.json", "data/sharded_gsm8k_train_400_selected.json"]

all_samples = []

for fn in synthetic_fns:
    with open(fn, "r") as f:
        data = json.load(f)
    all_samples += data

all_samples = [s for s in all_samples if len(s["shards"]) == 4]

print(Counter([sample["task"] for sample in all_samples]))
print(Counter([len(sample["shards"]) for sample in all_samples]))

with open(output_fn, "w") as f:
    json.dump(all_samples, f, indent=4)

Counter({'math': 66, 'code': 43})
Counter({4: 109})
